In [9]:
import torch

In [10]:
device = 'cuda'
checkpoint = "./checkpoints/nerfsynth/human_cuda/200000_net_ray_marching.pth"
saved_features = torch.load(checkpoint, map_location=device)

In [11]:
point_cloud = saved_features["neural_points.xyz"]

In [14]:
import struct
import numpy as np
def write_pointcloud(filename,xyz_points,rgb_points=None):

    """ creates a .pkl file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.zeros(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tostring(),rgb_points[i,1].tostring(),
                                        rgb_points[i,2].tostring())))
    fid.close()

In [16]:
write_pointcloud('./data_src/nerf/nerf_synthetic/human/pointcloud/67.ply', point_cloud.cpu().numpy())

/tmp/ipykernel_55029/1037235596.py:29: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  rgb_points[i,0].tostring(),rgb_points[i,1].tostring(),
/tmp/ipykernel_55029/1037235596.py:30: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  rgb_points[i,2].tostring())))


In [16]:
point_cloud[point_cloud[:,-1] < 0] = torch.zeros((133413, 3)).to(device)

In [19]:
saved_features["neural_points.xyz"] = point_cloud

In [20]:
torch.save(saved_features, checkpoint)